Using Yolov5


In [3]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_flag_yolov(image_path, model_path='yolov5s.pt'):
    """
    Uses YOLOv5 for object detection to identify flags in an image.
    Detects regions of interest (e.g., red and white bands) for flag identification.

    Args:
        image_path (str): Path to the image file.
        model_path (str): Path to the YOLOv5 model (default is 'yolov5s.pt').

    Returns:
        str: "Indonesia", "Poland", or "Unknown" based on detected flag.
    """
    try:
        # Load YOLOv5 model
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

        # Perform detection
        results = model(image_path)

        # Parse results
        detections = results.pandas().xyxy[0]  # Pandas DataFrame of results
        
        # Filter detections for flags (based on custom training or predefined classes)
        flag_detections = detections[detections['name'].str.contains("flag", case=False, na=False)]

        if flag_detections.empty:
            return "Unknown: No flags detected"

        # Load the image for further analysis
        image = cv2.imread(image_path)
        image_resized = cv2.resize(image, (640, 640))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Analyze detected regions to determine if it's Indonesia or Poland
        for _, row in flag_detections.iterrows():
            x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

            # Crop the detected flag region
            flag_region = image_rgb[y_min:y_max, x_min:x_max]
            
            # Resize for consistent analysis
            flag_region = cv2.resize(flag_region, (300, 200))

            # Split into top and bottom halves
            mid_line = flag_region.shape[0] // 2
            top_half = flag_region[:mid_line, :, :]
            bottom_half = flag_region[mid_line:, :, :]

            # Calculate mean color for each half
            top_mean_color = np.mean(top_half, axis=(0, 1))
            bottom_mean_color = np.mean(bottom_half, axis=(0, 1))

            # Determine flag based on color positions
            is_top_red = top_mean_color[0] > 150 and top_mean_color[1] < 100 and top_mean_color[2] < 100
            is_bottom_white = bottom_mean_color[0] > 180 and bottom_mean_color[1] > 100 and bottom_mean_color[2] > 180

            is_top_white = top_mean_color[0] > 150 and top_mean_color[1] > 100 and top_mean_color[2] > 100
            is_bottom_red = bottom_mean_color[0] > 180 and bottom_mean_color[1] < 180 and bottom_mean_color[2] < 100

            if is_top_red and is_bottom_white:
                return "Indonesia"
            elif is_top_white and is_bottom_red:
                return "Poland"

        return "Unknown: Flag type not determined"

    except Exception as e:
        return f"Error: {str(e)}"

# Test the YOLOv5-based flag detection function
if __name__ == "__main__":
    refined_flag_result = detect_flag_yolov(r"C:\Users\Vibha Narayan\OneDrive\Pictures\Screenshots\Poland Flag.png")
    print(refined_flag_result)

    refined_flag_result = detect_flag_yolov(r"C:\Users\Vibha Narayan\OneDrive\Pictures\Screenshots\Indonesian Flag.png")
    print(refined_flag_result)

    refined_flag_result = detect_flag_yolov(r"C:\Users\Vibha Narayan\OneDrive\Pictures\Screenshots\Indonesia Flag.png")
    print(refined_flag_result)


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Vibha Narayan/.cache\torch\hub\master.zip
YOLOv5  2024-12-21 Python-3.12.0 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\Vibha Narayan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Unknown: No flags detected


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Vibha Narayan/.cache\torch\hub\master.zip
YOLOv5  2024-12-21 Python-3.12.0 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\Vibha Narayan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Unknown: No flags detected


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Vibha Narayan/.cache\torch\hub\master.zip
YOLOv5  2024-12-21 Python-3.12.0 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\Vibha Narayan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Unknown: No flags detected


Using Yolov8

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

def detect_flag_with_yolov8(image_path):
    # Load the YOLOv8 model
    model = YOLO('yolov8n.pt')  # Replace with your custom model if trained

    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform object detection
    results = model.predict(image_rgb)

    # Extract bounding boxes and class labels
    boxes = results[0].boxes  # Boxes for the first image
    annotations = []
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].numpy()  # Bounding box coordinates
        cls = int(box.cls[0])  # Class index
        conf = float(box.conf[0])  # Confidence score
        annotations.append((x1, y1, x2, y2, cls, conf))

    # Analyze detected boxes for red and white regions
    red_boxes = []
    white_boxes = []

    for x1, y1, x2, y2, cls, conf in annotations:
        if conf < 0.2:  # Skip low-confidence detections
            continue
        # Assuming class 0 is red and class 1 is white (adjust as needed)
        if cls == 0:  # Red
            red_boxes.append((x1, y1, x2, y2))
        elif cls == 1:  # White
            white_boxes.append((x1, y1, x2, y2))

    if not red_boxes or not white_boxes:
        return "Unknown: Flag regions not detected"

    # Use the largest red and white regions for decision-making
    largest_red = max(red_boxes, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))
    largest_white = max(white_boxes, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))

    # Get the center Y-coordinates of the largest red and white regions
    red_y_center = (largest_red[1] + largest_red[3]) / 2
    white_y_center = (largest_white[1] + largest_white[3]) / 2

    # Determine the flag type
    if red_y_center < white_y_center:
        flag = "Indonesia"  # Red is above white
    else:
        flag = "Poland"  # White is above red

    # Draw bounding boxes on the image
    for x1, y1, x2, y2 in red_boxes:
        cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 3)  # Red bounding box
    for x1, y1, x2, y2 in white_boxes:
        cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3)  # White bounding box

    # Display the image
    plt.figure(figsize=(10, 5))
    plt.imshow(image_rgb)
    plt.title("Detected Flag")
    plt.axis('off')
    plt.show()

    return flag

# Test the function with your images
result1 = detect_flag_with_yolov8(r"C:\Users\Vibha Narayan\OneDrive\Pictures\Screenshots\Poland Flag.png")
print(result1)
result2 = detect_flag_with_yolov8(r"C:\Users\Vibha Narayan\OneDrive\Pictures\Screenshots\Indonesian Flag.png")
print(result2)
result3 = detect_flag_with_yolov8(r"C:\Users\Vibha Narayan\OneDrive\Pictures\Screenshots\Indonesia Flag.png")
print(result3)



0: 448x640 1 umbrella, 1 kite, 260.0ms
Speed: 20.3ms preprocess, 260.0ms inference, 16.0ms postprocess per image at shape (1, 3, 448, 640)
Unknown: Flag regions not detected

0: 448x640 1 bed, 180.9ms
Speed: 3.4ms preprocess, 180.9ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Unknown: Flag regions not detected

0: 448x640 (no detections), 181.6ms
Speed: 8.0ms preprocess, 181.6ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Unknown: Flag regions not detected
